<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aplicación-en-el-flujo-de-Machine-Learning" data-toc-modified-id="Aplicación-en-el-flujo-de-Machine-Learning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aplicación en el flujo de Machine Learning</a></span><ul class="toc-item"><li><span><a href="#Media-vs-Mediana" data-toc-modified-id="Media-vs-Mediana-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Media vs Mediana</a></span></li><li><span><a href="#Cómo-efectuar-la-imputación-en-ML" data-toc-modified-id="Cómo-efectuar-la-imputación-en-ML-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cómo efectuar la imputación en ML</a></span><ul class="toc-item"><li><span><a href="#El-concepto-fundamental" data-toc-modified-id="El-concepto-fundamental-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>El concepto fundamental</a></span></li><li><span><a href="#Lo-anterior-está-muy-bien-para-completar-los-datos-faltantes-en-el-Train-Set,-pero-cómo-proceder-con-los-valores-nulos-del-Test-Set-o-los-que-aparezcan-en-producción?" data-toc-modified-id="Lo-anterior-está-muy-bien-para-completar-los-datos-faltantes-en-el-Train-Set,-pero-cómo-proceder-con-los-valores-nulos-del-Test-Set-o-los-que-aparezcan-en-producción?-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Lo anterior está muy bien para completar los datos faltantes en el Train Set, pero cómo proceder con los valores nulos del Test Set o los que aparezcan en producción?</a></span></li><li><span><a href="#Qué-pasaría-si-justo-en-una-columna-o-variable-que-en--el-Train-Set-no-tenía-nulos-aparecen-valores-nulos-en-el-Test-Set-o-en-las-observaciones-nuevas-que-lleguen-en-producción?" data-toc-modified-id="Qué-pasaría-si-justo-en-una-columna-o-variable-que-en--el-Train-Set-no-tenía-nulos-aparecen-valores-nulos-en-el-Test-Set-o-en-las-observaciones-nuevas-que-lleguen-en-producción?-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Qué pasaría si justo en una columna o variable que en  el Train Set no tenía nulos aparecen valores nulos en el Test Set o en las observaciones nuevas que lleguen en producción?</a></span></li></ul></li><li><span><a href="#Ejemplo-1" data-toc-modified-id="Ejemplo-1-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Ejemplo 1</a></span></li></ul></li></ul></div>

![IES21](img/logo_ies.png)

# Situación Profesional 4: Preparación de los Datos
# Limpieza de los Datos - Imputación

## Aplicación en el flujo de Machine Learning

Ya hemos aprendido cómo imputar o asignar algún valor a los valores nulos que podamos tener en nuestro DataFrame, ahora vamos a ver cómo aplicarlo al flujo de Machine Learning. 

In [1]:
import numpy as np  
import pandas as pd  
from sklearn.impute import SimpleImputer  
from sklearn.compose import ColumnTransformer  

### Media vs Mediana  

Veamos el siguiente caso: 

In [2]:
dfA=pd.DataFrame([[5,3,"Si"],[8,8,'Si'],[10,np.nan,'No'],[2,5,'No'],[7,5,np.nan],[9,60,'Si']])
dfA.columns=['x1','x2','x3']
dfA

,x1,x2,x3
0,5,3.0,Si
1,8,8.0,Si
2,10,NaN,No
3,2,5.0,No
4,7,5.0,NaN
5,9,60.0,Si


Observemos los valores de la variable **x<sub>2</sub>**, podemos ver que nos falta un valor. Como x<sub>2</sub> es una variable **numérica** típicamente se le suele imputar:   

- la **media** o promedio de los valores de x<sub>2</sub>
- la **mediana** de los valores de x<sub>2</sub>

Si utiizamos la media o promedio el valor a imputar sería:   

$$ \frac{3+8+5+5+60}{5} = \frac{81}{5} = 16.2$$

Observe que el resultado de la media se ha visto influido **fuertemente** por la presencia del **outlier de valor 60**.    

> **Si sospechamos o hemos observado la presencia de outliers convendrá utlizar la mediana que es inmume a la presencia de outliers:**  

Para calcular la mediana, debemos ordenar las observaciones de menor a mayor y elegimos la que se encuentra en el medio. Como las observaciones son:    

$$ 3,5,5,8,60 $$

La observación que se encuentra en la posicón central tiene valor 5, entonces la mediana es 5. Como podemos observar el valor que tomaba el outlier no tuvo influencia, podría haber valido 500 y el valor de la mediana hubiera seguido siendo 5.  

Esto no significa que siempre convenga usar la mediana. En general se suele utilizar la media y si se detectan outliers, entonces se utiliza la mediana. 


### Cómo efectuar la imputación en ML

Antes que nada, aclaremos una cosa: asumimos que la variable target, y, no tiene valores faltantes (ya que si los tuviera la observación no nos serviría para aprender mediante Aprendizaje Supervisado)!  

Por ende   

> no efectuaremos imputaciones a los valores del target!

#### El concepto fundamental

Si nuestro interés  es efectuar la imputación, cualquiera sea (media, mediana, moda, o pronóstico, etc), para luego aplicar a los datos algún modelo de pronóstico de Machine Learning, entonces tenemos que tener en cuenta el **concepto más importante** de todos con respecto a la preparación de los datos (y cuyo desconocimiento es fruto de errores):   

> Debemos imputar **después** de haber efectuado la división entre el Train Set y el Test Set, y **obteniendo los valores de media, mediana o moda sólo de las observaciones del Train Set**.

No olvidemos que el Test Set representa a las observaciones desconocidas, digamos, las que están por venir en el futuro para que pronostiquemos, por lo tanto **no pueden participar en el cálculo de los valores con los cuales imputaremos a los nulos!**


#### Lo anterior está muy bien para completar los datos faltantes en el Train Set, pero cómo proceder con los valores nulos del Test Set o los que aparezcan en producción?

Seguramente en el Test Set algunas observaciones tendrán valores nulos y quizá cuando estemos en producción también llegarán nuevas observaciones con nulos, cómo debemos proceder en estos casos?  
Quizá el primer impulso sea rechazarlos, capturarlos en alguna línea de programa y sacar un cartel que diga que no se puede pronosticar, pero no es necesario llegar a tanto, en realidad sí podremos pronosticar con ellos:  

> **A los valores nulos del Test Set o los que pudieran llegar en producción les imputaremos el valor elegido (media, mediana, moda, pronóstico) obtenido con anterioridad en el Train Set.**



#### Qué pasaría si justo en una columna o variable que en  el Train Set no tenía nulos aparecen valores nulos en el Test Set o en las observaciones nuevas que lleguen en producción?

Esta una una duda muy interesante, después de todo la división entre el Train Set y el Test Set se hace generalmente por azar, por ende podría darse que, por azar, en una columna al Train Set no le tocaran nulos y que los mismos quedaran todos en el Test Set.   

Sin embargo, la respuesta es simple, no debe pasar nada raro, porque:  

> **Definiremos imputaciones para todas las columnas o variables aunque no tengamos valores nulos en ellas!**

### Ejemplo 1

Supongamos que tenemos el siguiente DataFrame df, generemos Train Set y Test Set sin valores nulos, para luego usarlos en algún modelo de ML.

In [4]:
df=pd.DataFrame([[4,30,'Alto',0.2,'Si'],[8,np.nan,'Bajo',5,'No'],[0,50,'Alto',np.nan,'No'],
                 [np.nan,20,'Alto',0.5,'Si'],[6,25,'Alto',0.8,'No'], [10,np.nan,'Bajo',0.9,'Si'],
                 [2,45,np.nan,np.nan,'No'],[np.nan,10,'Alto',0.7,'Si'],[3,55,'Alto',0.6,'No'],[5,75,np.nan,0.1,'Si'],
                 [np.nan,25,'Bajo',np.nan,'No'],[1,np.nan,'Alto',np.nan,'Si'],[np.nan,25,np.nan,0.5,'No'],
                 [np.nan,65,'Alto',0.3,'Si'],[np.nan,np.nan,'Alto',0.5,'Si'],[10,np.nan,np.nan,0.9,'Si'],
                 [np.nan,88,'Bajo',0.4,'Si'],[4,30,'Alto',0.2,'Si'],[8,np.nan,'Bajo',5,'No'],[1,np.nan,'Alto',np.nan,'Si'],
                 [np.nan,25,np.nan,0.5,'No'],[2,45,np.nan,np.nan,'No'],[np.nan,10,'Alto',0.7,'Si'] ])
df.columns=['x1','x2','x3','x4','y']
df


,x1,x2,x3,x4,y
0,4.0,30.0,Alto,0.2,Si
1,8.0,NaN,Bajo,5.0,No
2,0.0,50.0,Alto,NaN,No
3,NaN,20.0,Alto,0.5,Si
4,6.0,25.0,Alto,0.8,No
5,10.0,NaN,Bajo,0.9,Si
6,2.0,45.0,NaN,NaN,No
7,NaN,10.0,Alto,0.7,Si
8,3.0,55.0,Alto,0.6,No
9,5.0,75.0,NaN,0.1,Si


Para no cometer errores al imputar, ya quitemos la columna target ,y, y hagamos una división inicial en Train y Test usemos 80/20 y 123 para randomizar:

In [6]:
from sklearn.model_selection import train_test_split
X=df.drop(columns=['y'])
X_train, X_test, y_train, y_test = train_test_split(X, df['y'], test_size=0.2, random_state=123)

Veamos qué tenemos en X_train

In [5]:
X_train

,x1,x2,x3,x4
8,3.0,55.0,Alto,0.6
7,NaN,10.0,Alto,0.7
11,1.0,NaN,Alto,NaN
4,6.0,25.0,Alto,0.8
3,NaN,20.0,Alto,0.5
22,NaN,10.0,Alto,0.7
12,NaN,25.0,NaN,0.5
15,10.0,NaN,NaN,0.9
9,5.0,75.0,NaN,0.1
16,NaN,88.0,Bajo,0.4


In [6]:
X_test

,x1,x2,x3,x4
5,10.0,NaN,Bajo,0.9
19,1.0,NaN,Alto,NaN
20,NaN,25.0,NaN,0.5
18,8.0,NaN,Bajo,5.0
14,NaN,NaN,Alto,0.5


Nosotros vamos a asignar el promedio en las variables numéricas y la moda en las restantes:

In [7]:
from sklearn.impute import SimpleImputer  
from sklearn.compose import ColumnTransformer 

cols_num=X_train.select_dtypes(np.number).columns  
cols_cat=X_train.select_dtypes(include=['object','bool']).columns

t1=('imputador_num', SimpleImputer(strategy='mean'), cols_num)  
t2=('imputador_cat', SimpleImputer(strategy='most_frequent'), cols_cat) 

transformador_columnas = ColumnTransformer(transformers=[t1,t2],remainder='passthrough')  


A nuestro transformador de columnas lo **entrenamos en el X_train**

In [8]:
transformador_columnas.fit(X_train);

Ahora aplicamos nuestro transformador de columnas al X_train para imputar sus nulos

In [9]:
Train_transformado = transformador_columnas.transform(X_train)
Train_transformado

array([[3.0, 55.0, 0.6, 'Alto'],
       [4.090909090909091, 10.0, 0.7, 'Alto'],
       [1.0, 39.86666666666667, 0.8384615384615385, 'Alto'],
       [6.0, 25.0, 0.8, 'Alto'],
       [4.090909090909091, 20.0, 0.5, 'Alto'],
       [4.090909090909091, 10.0, 0.7, 'Alto'],
       [4.090909090909091, 25.0, 0.5, 'Alto'],
       [10.0, 39.86666666666667, 0.9, 'Alto'],
       [5.0, 75.0, 0.1, 'Alto'],
       [4.090909090909091, 88.0, 0.4, 'Bajo'],
       [4.0, 30.0, 0.2, 'Alto'],
       [8.0, 39.86666666666667, 5.0, 'Bajo'],
       [4.090909090909091, 25.0, 0.8384615384615385, 'Bajo'],
       [4.0, 30.0, 0.2, 'Alto'],
       [2.0, 45.0, 0.8384615384615385, 'Alto'],
       [2.0, 45.0, 0.8384615384615385, 'Alto'],
       [0.0, 50.0, 0.8384615384615385, 'Alto'],
       [4.090909090909091, 65.0, 0.3, 'Alto']], dtype=object)

y también **lo aplicamos al X_test** para imputar sus nulos:

In [10]:
Test_transformado = transformador_columnas.transform(X_test)
Test_transformado

array([[10.0, 39.86666666666667, 0.9, 'Bajo'],
       [1.0, 39.86666666666667, 0.8384615384615385, 'Alto'],
       [4.090909090909091, 25.0, 0.5, 'Alto'],
       [8.0, 39.86666666666667, 5.0, 'Bajo'],
       [4.090909090909091, 39.86666666666667, 0.5, 'Alto']], dtype=object)

Tanto y_train como y_test no debían sufrir imputaciones, así que seguirán siendo las mismas de antes. 

Es decir que tenemos a  

- X_train_transformado
- X_test_transformado
- y_train, 
- y_test

Listos para aplicar a cualquier modelo de Machine Learning.


Por otro lado, si estuvierámos en producción, a los datos de entrada deberíamos aplicarle nuestro transformador de columnas entrenado en todo el X, para lo cual deberíamos crear el nuevo transformador de columnas usando **todas** las observaciones.

~~~
transformador_columnas_produccion.transform(nueva_observación)
~~~

Cuidado con el orden de las columnas en el resultado!!!

Si es necesario, convertimos estos dos últimos arrays en DataFrames, pero hay que tener cuidado con un detalle:  
    
el orden de las columnas

Cada uno de los imputadores las ordenó ubicando primero las que afectó y ésto lo hizo según como las tenemos en cols_num y cols_cat. 

Para no cometer errores vamos a crear una lista con el orden de las columnas en ellos en el orden en que procesamos:

~~~
cols_orden=cols_num.tolist() + cols_cat.tolist()
~~~

In [11]:
# Si es necesario convertimos estos dos últimos arrays en DataFrames
# Cuidado con el orden de las columnas. Cuando las hemos transformado NO se preserva su orden. 

cols_orden=cols_num.tolist() + cols_cat.tolist()
X_train_transformado=pd.DataFrame(Train_transformado, columns=cols_orden) 
X_test_transformado=pd.DataFrame(Test_transformado, columns=cols_orden)
                                

display(X_train_transformado)
display(X_test_transformado)

,x1,x2,x4,x3
0,3,55,0.6,Alto
1,4.09091,10,0.7,Alto
2,1,39.8667,0.838462,Alto
3,6,25,0.8,Alto
4,4.09091,20,0.5,Alto
5,4.09091,10,0.7,Alto
6,4.09091,25,0.5,Alto
7,10,39.8667,0.9,Alto
8,5,75,0.1,Alto
9,4.09091,88,0.4,Bajo


,x1,x2,x4,x3
0,10,39.8667,0.9,Bajo
1,1,39.8667,0.838462,Alto
2,4.09091,25,0.5,Alto
3,8,39.8667,5,Bajo
4,4.09091,39.8667,0.5,Alto
